# Test the Takahasi-Takami method for calculating the M/H2/n system

Idea of this algorithm was first proposed by Takahashi and Takami

>Takahashi Y., Takami Y. A Numerical Method for the Steady-State
Probabilities of a GI/G/c Queuing System in a General Class // J. of the
Operat. Res. Soc. of Japan. 1976. — v. 19, no. 2. — P. 147–157.

This method allows to calculate the system for an arbitrary coefficient of variation of service time. When the coefficient of variation of service time < 1, parameters of the approximating H2-distribution are complex, which does not prevent obtaining meaningful results.
For verification we use simulation.


#### Import necessary modules

In [ ]:
from most_queue.sim.queueing_systems.fifo import QueueingSystemSimulator
import most_queue.rand_distribution as rd
from most_queue.theory.queueing_systems.fifo.mgn_takahasi import MGnCalc
from most_queue.general.tables import probs_print, times_print

import time
import math


#### Set paramrameters for the simulation and calculation 

In [2]:
n = 3  # number of channels
l = 1.0  # intensity of jobs arrival
ro = 0.8  # load factor
b1 = n * ro / l  # average service time
num_of_jobs = 800000  # number of jobs required for the simulation
b_coev = [0.42, 1.5]  # values of the coefficient of variation of service time

#### Running the simulation and calcalculation for values of the coefficient of variation of service time (0.42 and 1.5):

In [3]:
for k in range(len(b_coev)):
    #  Set parameters for the simulation and calculation
    b = [0.0] * 3
    alpha = 1 / (b_coev[k] ** 2)
    b[0] = b1
    b[1] = math.pow(b[0], 2) * (math.pow(b_coev[k], 2) + 1)
    b[2] = b[1] * b[0] * (1.0 + 2 / alpha)

    #  run Takahasi-Takami method for M/Gn/1 queue model
    tt_start = time.process_time()

    tt = MGnCalc(n, l, b)
    tt.run()
    # Get numeric results from Takahasi-Takami method
    p_tt = tt.get_p()
    v_tt = tt.get_v()
    tt_time = time.process_time() - tt_start
    
    num_of_iter = tt.num_of_iter_

    im_start = time.process_time()

    qs = QueueingSystemSimulator(n)

    # Set the arrival process. M - exponential with intensity l
    qs.set_sources(l, 'M')

    # Set servers with Gamma distribution
    gamma_params = rd.Gamma.get_params([b[0], b[1]])
    qs.set_servers(gamma_params, 'Gamma')

    # Run the simulation
    qs.run(num_of_jobs)

    # Get results from simulation
    p = qs.get_p()
    v_sim = qs.v
    im_time = time.process_time() - im_start

    print("\nComparison of results calculated by the Takahasi-Takami method and Simulation.\n"
          f"Sim - M/Gamma/{n:^2d}\nTakahasi-Takami - M/H2/{n:^2d}"
          "with complex parameters\n"
          f"Load coefficient: {ro:^1.2f}\n"
          f"Variation coefficient of service time: {b_coev[k]:^1.2f}\n"
          f"Number of iterations in the Takahasi-Takami algorithm: {num_of_iter:^4d}\n"
          f"Time taken by the Takahasi-Takami algorithm: {tt_time:^5.3f} s\n"
          f"Time taken by Sim: {im_time:^5.3f} s")
    probs_print(p, p_tt, 10)
    times_print(v_sim, v_tt, is_w=False)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]8000/800000:   1%|          | 1/100 [00:00<00:06, 14.26it/s]8000/800000:   2%|▏         | 2/100 [00:00<00:06, 14.12it/s]16000/800000:   2%|▏         | 2/100 [00:00<00:06, 14.12it/s]24000/800000:   3%|▎         | 3/100 [00:00<00:06, 14.12it/s]24000/800000:   4%|▍         | 4/100 [00:00<00:07, 12.51it/s]32000/800000:   4%|▍         | 4/100 [00:00<00:07, 12.51it/s]40000/800000:   5%|▌         | 5/100 [00:00<00:07, 12.51it/s]40000/800000:   6%|▌         | 6/100 [00:00<00:07, 12.65it/s]48000/800000:   6%|▌         | 6/100 [00:00<00:07, 12.65it/s]56000/800000:   7%|▋         | 7/100 [00:00<00:07, 12.65it/s]56000/800000:   8%|▊         | 8/100 [00:00<00:07, 12.82it/s]64000/800000:   8%|▊         | 8/100 [00:00<00:07, 12.82it/s]72000/800000:   9%|▉         | 9/100 [00:00<00:07, 12.82it/s]72000/800000:  10%|█         | 10/100 [00:00<00:07, 12.11it/s]80000/800000:  10%|█         | 10/100 [00:00<00:07, 12.11it/s]88000/800000:  11%|█         | 11/100 [00:00<0

Simulation is finished


Comparison of results calculated by the Takahasi-Takami method and Simulation.
Sim - M/Gamma/3 
Takahasi-Takami - M/H2/3 with complex parameters
Load coefficient: 0.80
Variation coefficient of service time: 0.42
Number of iterations in the Takahasi-Takami algorithm:  10 
Time taken by the Takahasi-Takami algorithm: 0.018 s
Time taken by Sim: 9.065 s
------------------------------------
      Probabilities of states       
------------------------------------
 #  |    Numeric    |      Sim      
------------------------------------
 0  |   0.053201    |   0.052577    
 1  |    0.13317    |    0.13268    
 2  |    0.17406    |    0.17509    
 3  |    0.17103    |    0.16762    
 4  |     0.136     |    0.13533    
 5  |   0.099993    |    0.1001     
 6  |   0.071056    |   0.071302    
 7  |    0.04971    |   0.050199    
 8  |   0.034526    |   0.035366    
 9  |   0.023899    |   0.024872    
------------------------------------

Initial moments of soujorn tim

Job served:    | 0/100 [00:00<?, ?it/s]8000/800000:   1%|          | 1/100 [00:00<00:07, 12.97it/s]8000/800000:   2%|▏         | 2/100 [00:00<00:07, 12.55it/s]16000/800000:   2%|▏         | 2/100 [00:00<00:07, 12.55it/s]24000/800000:   3%|▎         | 3/100 [00:00<00:07, 12.55it/s]24000/800000:   4%|▍         | 4/100 [00:00<00:07, 12.59it/s]32000/800000:   4%|▍         | 4/100 [00:00<00:07, 12.59it/s]40000/800000:   5%|▌         | 5/100 [00:00<00:07, 12.59it/s]40000/800000:   6%|▌         | 6/100 [00:00<00:07, 12.16it/s]48000/800000:   6%|▌         | 6/100 [00:00<00:07, 12.16it/s]56000/800000:   7%|▋         | 7/100 [00:00<00:07, 12.16it/s]56000/800000:   8%|▊         | 8/100 [00:00<00:07, 12.40it/s]64000/800000:   8%|▊         | 8/100 [00:00<00:07, 12.40it/s]72000/800000:   9%|▉         | 9/100 [00:00<00:07, 12.40it/s]72000/800000:  10%|█         | 10/100 [00:00<00:07, 12.46it/s]80000/800000:  10%|█         | 10/100 [00:00<00:07, 12.46it/s]88000/800000:  11%|█         | 11/100 [00:00<0

Simulation is finished


Comparison of results calculated by the Takahasi-Takami method and Simulation.
Sim - M/Gamma/3 
Takahasi-Takami - M/H2/3 with complex parameters
Load coefficient: 0.80
Variation coefficient of service time: 1.50
Number of iterations in the Takahasi-Takami algorithm:  26 
Time taken by the Takahasi-Takami algorithm: 0.041 s
Time taken by Sim: 9.394 s
------------------------------------
      Probabilities of states       
------------------------------------
 #  |    Numeric    |      Sim      
------------------------------------
 0  |   0.058345    |   0.057806    
 1  |    0.13622    |    0.13495    
 2  |    0.15252    |    0.15477    
 3  |    0.10273    |   0.097882    
 4  |    0.07841    |   0.077205    
 5  |   0.064082    |   0.064347    
 6  |   0.054151    |   0.054398    
 7  |   0.046478    |   0.046695    
 8  |   0.040177    |   0.040742    
 9  |   0.034845    |   0.035463    
------------------------------------

Initial moments of soujorn tim